In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install bitsandbytes accelerate

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-pq91y78g did not run successfully.
  │ exit code: 128
  ╰─> [5 lines of output]
      error: RPC failed; curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
      error: 2167 bytes of body are still expected
      fetch-pack: unexpected disconnect while reading sideband packet
      fatal: early EOF
      fatal: fetch-pack: invalid index-pack output
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-pq91y78g did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
%env HF_HOME=/workspace/hfcache

env: HF_HOME=/workspace/hfcache


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

In [9]:
MODEL_ID = "TheBloke/Nous-Hermes-2-Mixtral-8x7B-DPO-GGUF"

In [10]:
def load_model_and_tokenizer():
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        fsdp_plugin = FullyShardedDataParallelPlugin(
            state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
            optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
        )
        accelerator = Accelerator(fsdp_plugin=fsdp_plugin)
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, quantization_config=quantization_config, device_map="auto")
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    return model, tokenizer

def create_text_generation_pipeline(model, tokenizer):
    generate_text = pipeline(
        model=model,
        tokenizer=tokenizer,
        return_full_text=True,
        task="text-generation",
        do_sample=True,
        temperature=0.2,
        top_p=0.15,
        top_k=0,
        max_new_tokens=8192,
        repetition_penalty=1.1,
    )
    return generate_text

In [11]:
model, tokenizer = load_model_and_tokenizer()
generate_text = create_text_generation_pipeline(model, tokenizer)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


OSError: TheBloke/Nous-Hermes-2-Mixtral-8x7B-DPO-GGUF does not appear to have a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.

In [ ]:
initial_context = [
    {"role": "user", "content": "You are a creative writer who can come up with inventive and awesome stories."},
    {"role": "assistant", "content": "Thanks for this additional context. I will certainly follow this instructions."},
    ]

def update_messages(msgs_, input_):
    msg_ = [{"role": "user", "content": input_}]
    return msgs_ + msg_

In [ ]:
query = 'Give me a short mythical story where the main theme is grief of a lossed one'
messages = update_messages(initial_context, query)

In [ ]:
tmplt_messages = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = generate_text(tmplt_messages)
answer = response[0]["generated_text"]
print(answer)